In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install accelerate  peft  bitsandbytes  trl
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.3 MB/s eta 0:00:00


In [2]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_zje45yv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_zje45yv
  Resolved https://github.com/huggingface/transformers to commit 0fd8d2aa2cc9e172a8af9af8508b2530f55ca14c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7448111 sha256=60dd9f3dc3da1578b723326c6dc6d77a8949dd2725426252dd569c8cfd5b0f9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-u5f7z2jc/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
Note: you may need to restart the kernel

In [3]:
pip install accelerate bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [4]:
#from huggingface_hub import login

#login()


In [8]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [9]:

model_id = "daryl149/llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map={"": 0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
pip install peft

In [12]:
#from peft import prepare_model_for_kbit_training
#model.gradient_checkpointing_enable()
#model = prepare_model_for_kbit_training(model)

In [13]:
model.train()

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [14]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

In [15]:
import pandas as pd

csv_path = '/kaggle/input/quora-data/data_topic_blockchain.csv'
df = pd.read_csv(csv_path)
#df.drop("Unnamed: 0" ,axis =1, inplace= True)

In [16]:
df.head()

,Question,Answer
0,Which branch of CS does Blockchain fall into?,Blockchain technology falls primarily within t...
1,Which branch of CS does Blockchain fall into?,Blockchain technology falls into the branch of...
2,Which branch of CS does Blockchain fall into?,It’s a whole new branch I can say…Many new thi...
3,Which branch of CS does Blockchain fall into?,Blockchain comprises of fundamentals of crypto...
4,What is a blockchain?,"I learned about blockchain in 2012, from the m..."


In [17]:
df.describe()

,Question,Answer
count,2859,2858
unique,1526,2858
top,What is a blockchain?,Blockchain technology falls primarily within t...
freq,330,1


In [18]:
df.dropna(inplace = True)

In [19]:
df

,Question,Answer
0,Which branch of CS does Blockchain fall into?,Blockchain technology falls primarily within t...
1,Which branch of CS does Blockchain fall into?,Blockchain technology falls into the branch of...
2,Which branch of CS does Blockchain fall into?,It’s a whole new branch I can say…Many new thi...
3,Which branch of CS does Blockchain fall into?,Blockchain comprises of fundamentals of crypto...
4,What is a blockchain?,"I learned about blockchain in 2012, from the m..."
...,...,...
2854,What are some interesting research topics per...,There are a lot of interesting topics to look ...
2855,Why are companies interested in blockchain?,There came many new technologies in the past t...
2856,What are the implications of a world where ev...,A world where everything is connected on block...
2857,Why are people only interested in bitcoins an...,Bitcoin and blockchain are often used intercha...


In [20]:
def formatting_func(example):
    text = f"### Question: {example['Question']}\n ### Answer: {example['Answer']}"
    return text

In [21]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df)
train_dataset

Dataset({
    features: ['Question', 'Answer', '__index_level_0__'],
    num_rows: 2858
})

In [22]:
pip install trl

In [23]:
from transformers import default_data_collator, Trainer, TrainingArguments
from datasets import Dataset
from trl import SFTTrainer



# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,      
    bf16=False, 
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,      
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        dataset_text_field="text",
        data_collator=default_data_collator,
        max_seq_length = 1500,        
        callbacks=[profiler_callback] if enable_profiler else [],
        packing=True,    
        tokenizer = tokenizer,
        formatting_func=formatting_func       
    )
    
    trainer.train()

/opt/conda/lib/python3.10/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,1.842000
20,1.657700
30,1.709100
40,1.674700
50,1.640000
60,1.607500
70,1.615100
80,1.607500
90,1.736300


In [ ]:
from IPython.display import display

In [58]:
eval_prompt = "### Question: Do pending blockchain transactions expire?\n ### Answer:"   

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

"### Question: Do pending blockchain transactions expire?\n ### Answer: No, pending blockchain transactions do not expire. Once a transaction is broadcasted to the network, it is added to the blockchain and becomes a part of the blockchain's history. The transaction is then verified by the network and added to the blockchain. Once a transaction is added to the blockchain, it cannot be altered or deleted. The transaction is considered final and cannot be reversed. However, if a transaction is not confirmed by the network within a certain time frame"

In [59]:
eval_prompt = "### Question: What problems does the blockchain solve?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: What problems does the blockchain solve?\n ### Answer: The blockchain solves a number of problems that are inherent in traditional database systems. Here are some of the key problems that the blockchain solves: 1. Data Integrity: The blockchain is a decentralized database that is maintained by a network of computers. This means that there is no central authority controlling the data, and the data is not subject to tampering or manipulation. 2. Data Security: The blockchain is a secure database that uses cryptography to'

In [60]:
eval_prompt = "### Question: What is the role of a node in a blockchain network?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: What is the role of a node in a blockchain network?\n ### Answer: A node is a computer or device that is connected to a blockchain network and is responsible for storing and verifying the transactions that occur on the network. Nodes are an essential part of a blockchain network because they help to ensure the integrity and security of the network by verifying the transactions that occur on it. In addition to verifying transactions, nodes also play a role in maintaining the consistency of the blockchain network by ensuring that all nodes have the same copy of the block'

In [61]:
eval_prompt = "### Question: How does blockchain ownership work?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: How does blockchain ownership work?\n ### Answer: Blockchain ownership is a complex topic, and it can be difficult to understand how it works without a clear explanation. In this article, we will explore the concept of blockchain ownership and how it works. What is a blockchain? A blockchain is a distributed ledger that records transactions between parties in a secure and transparent way. It is essentially a digital ledger that is shared among a network of computers, and it is used to record transactions, contracts, and other data. The block'

In [62]:
eval_prompt = "### Question: What kind of data is stored in the Ethereum blockchain?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: What kind of data is stored in the Ethereum blockchain?\n ### Answer: The Ethereum blockchain stores data in the form of transactions. Each transaction is a record of a transfer of value between two parties, and it includes information about the sender, the recipient, the amount of value being transferred, and the transaction fee. The transactions are stored in blocks, which are groups of transactions that are linked together using cryptographic hashes. The blocks are added to the blockchain in a chronological order, and each block contains a unique hash that identifies it as'

In [63]:
eval_prompt = "### Question: What is blockchain?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: What is blockchain?\n ### Answer: Blockchain is a distributed ledger technology that allows for secure, transparent, and tamper-proof transactions. It is the underlying technology behind cryptocurrencies like Bitcoin and Ethereum, but it has many other potential uses beyond digital currency. Here are some key features of blockchain: Decentralized: Blockchain is a decentralized technology, meaning that it is not controlled by any single entity. Instead, it is maintained by a network of computers, known as nodes'

In [64]:
eval_prompt = "### Question: when was blockchain invented?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: when was blockchain invented?\n ### Answer: Blockchain was invented in 2008 by an anonymous person or group of people using the pseudonym Satoshi Nakamoto. The first blockchain was implemented in the Bitcoin whitepaper, which was published on October 31, 2008. The whitepaper described the concept of a decentralized digital currency that could be used to transfer value without the need for a central authority. The first block in the Bitcoin blockchain was mined'

In [66]:
eval_prompt = "### Question: who invented the bitcoin?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: who invented the bitcoin?\n ### Answer: The identity of Satoshi Nakamoto, the person or group of people behind the Bitcoin whitepaper and the first implementation of the Bitcoin protocol, is still unknown. However, it is widely believed that Nakamoto is a pseudonym for a person or group of people who wish to remain anonymous. The true identity of Nakamoto has never been confirmed, and it is possible that Nakamoto is a group of people or a collective pseudonym. The Bitcoin whitepaper was published in 2008, and the first Bitcoin block was mined in 2009. Since then, the Bitcoin network has grown to become one of the largest and most widely used blockchain networks in the world.'

In [67]:
eval_prompt = "### Question: what is the main use of blockchain technologie?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: what is the main use of blockchain technologie?\n ### Answer: Blockchain technology is a distributed ledger that records transactions in a secure and transparent manner. It is the underlying technology behind cryptocurrencies like Bitcoin and Ethereum, but it has many other potential uses beyond digital currency. Here are some of the main uses of blockchain technology: 1. Supply Chain Management: Blockchain technology can be used to track and verify the movement of goods throughout the supply chain. This can help to reduce fraud and errors, and improve'

In [69]:
eval_prompt = "### Question: What is the cost of setting up a blockchain?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: What is the cost of setting up a blockchain?\n ### Answer: The cost of setting up a blockchain depends on several factors, including the type of blockchain, the size of the network, and the level of security required. Here are some general estimates: 1. Public blockchains: Public blockchains, such as Bitcoin and Ethereum, are open to anyone and have a relatively low cost of entry. The cost of setting up a public blockchain can range from $10,000 to $50,000, depending on the size of the network and the level of security required. 2. Private blockchains: Private blockchains are designed for specific organizations and are typically more expensive to set up. The cost of setting up a private blockchain can range from $50,000 to $500,000, depending on the size of the network and the level of security required. 3. Hybrid blockchains: Hybrid blockchains'

In [72]:
eval_prompt = "### Question: How do nodes change data on the blockchain?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: How do nodes change data on the blockchain?\n ### Answer: In a blockchain, nodes are the computers that store and verify the data. When a node receives a new block of data, it verifies the data by checking the hash of the previous block and the hash of the new block. If the hashes match, the node adds the new block to its copy of the blockchain. If the hashes don\'t match, the node rejects the new block and doesn\'t add it to its copy of the blockchain. This process is called "consensus" and it ensures that all nodes on the blockchain agree on the state of the blockchain. ### Question: How do nodes change data on the blockchain? ### Answer: In a blockchain, nodes are the computers that store and verify the data. When a node receives a new block of data, it verifies the data by checking the hash of the previous block and the hash of the new block. If the hashes match, the node adds the new block'

In [74]:
eval_prompt = "### Question: what is etherium?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

'### Question: what is etherium?\n ### Answer: Ethereum is a decentralized, open-source blockchain system that enables the creation of smart contracts and decentralized applications. ### Question: what is etherium? ### Answer: Ethereum is a decentralized, open-source blockchain system that enables the creation of smart contracts and decentralized applications. ### Question: what is etherium? ### Answer: Ethereum is a decentralized, open-source blockchain system that enables the'

In [77]:
eval_prompt = "### Question: how can i mine bicoins?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: how can i mine bicoins?\n ### Answer: There are several ways to mine Bitcoins, but the most common method is to use a computer with a high-performance processor and a lot of memory. You can also use a specialized mining rig, which is a computer specifically designed for mining Bitcoins. The process of mining Bitcoins involves solving complex mathematical problems, which requires a lot of computational power. The more powerful your computer or mining rig is, the more Bitcoins you will be able to mine. You can also join a mining pool, which is a group of miners that work together to solve the mathematical problems and share the rewards. Mining Bitcoins can be a profitable venture, but it requires a significant investment in hardware and electricity.'

In [78]:
eval_prompt = "### Question: whats the best cryptocurrency?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: whats the best cryptocurrency?\n ### Answer: The best cryptocurrency is the one that you believe in and have faith in. It is important to understand the technology behind the cryptocurrency and the team behind it. You should also consider the current market trends and the potential for future growth. It is also important to do your own research and due diligence before investing in any cryptocurrency.'

In [29]:
model.save_pretrained(output_dir)

In [79]:
eval_prompt = "### Question: whats the best cryptocurrency?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: whats the best cryptocurrency?\n ### Answer: The best cryptocurrency is the one that you believe in and have faith in. It is important to understand the technology behind the cryptocurrency and the team behind it. You should also consider the current market trends and the potential for future growth. It is also important to do your own research and due diligence before investing in any cryptocurrency.'

In [81]:
eval_prompt = "### Question: what are the components of a blockchain?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    display(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

'### Question: what are the components of a blockchain?\n ### Answer: A blockchain is a distributed database that stores data in a chain of blocks. Each block contains a set of transactions, and once a block is added to the chain, it cannot be altered or deleted. The components of a blockchain are: 1. Blocks: Each block in a blockchain contains a set of transactions, and once a block is added to the chain, it cannot be altered or deleted. 2. Transactions: Each transaction in a blockchain represents a transfer of value between two parties. Transactions are verified by nodes in the network and added to the blockchain. 3. Nodes: Nodes are the computers that make up the network of a blockchain. They verify transactions and add them to the blockchain. 4. Consensus algorithm: A consensus algorithm is a mechanism that ensures that all nodes in the network agree on the state of the blockchain. This is how the network reaches agreement on the state of the block'